<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/Wording_for_ChatBot_(OPEN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
### 初回は、これを実行後に、上のメニューの 「ランタイム」ー「ランタイムの再起動」 を実行してください。
!pip install git+https://github.com/ramsrigouthamg/Questgen.ai

In [ ]:
### 上のメニューの 「ランタイム」ー「ランタイムの再起動」 を実行後に、これを実行してください。
!pip install sense2vec==1.0.2

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
!python -m nltk.downloader universal_tagset
!python -m spacy download en 

# Download Sense2vec wordvectors for generation of multiple choices

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz

In [ ]:
!tar -xvf  s2v_reddit_2015_md.tar.gz

In [ ]:
!ls s2v_old

In [ ]:
## これを実行しないと、＂from Questgen import main＂でエラーになった
!python -m nltk.downloader stopwords

# Data Read

In [ ]:
import os
import time
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/ChatBot_OPEN_0325.csv', sep=',', encoding='cp932')
_data

In [ ]:
data = _data[_data.columns.values[0:3]]
data

# Paraphrasing Questions

In [ ]:
from pprint import pprint
from Questgen import main
qg = main.QGen()

In [ ]:
!pip3 install git+https://github.com/alainrouillon/py-googletrans@feature/enhance-use-of-direct-api

In [ ]:
## 無料タイプ。利用しません。
#from googletrans import Translator
#Trans = Translator(service_urls=['translate.googleapis.com']) ## https://qiita.com/_yushuu/items/83c51e29771530646659

#def trans_google(qes, src='ja', to='en'):
#  return Trans.translate(qes, src=src ,dest=to).text

In [ ]:
from google.cloud import translate_v3beta1 as translate
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/Colab Notebooks/key/MimbleWimble-9c117f7eef20.json'
Trans = translate.TranslationServiceClient()

In [ ]:
def Paraphrasing(data):
  data_len = len(data)
  Answer = []
  for l, (idx, row) in enumerate(data.iterrows()):
    print(f'\n{l + 1} / {data_len}')
    No  = row[0]
    Qes = row[1]
    Ans = row[2]
    try:
      Answer.append(['{}_{:02x}'.format(No, 0), Qes, Ans])
      #_enQ   = trans_google(Qes, src='ja', to='en')　　　　　　　　　　　　　　　　　　　　　　　　　　## 無料タイプ。
      _enQ   = trans_googleCloud([Qes], src='ja', to='en-US')
      _enQes = paraphrasing_word(_enQ)
      enQes  = _enQes['Paraphrased Questions']
    
      _jpQes = []
      for wd in enQes:
        _qes = wd.split(':')[1].strip()
        #jpQes = trans_google(_qes, src='en', to='ja')　　　　　　　　　　　　　　　　　　　　　　　　## 無料タイプ。
        _jpQes.append(trans_googleCloud([_qes], src='en-US', to='ja'))
      _jpQes = set(_jpQes)
      print('jpQes LEN  :  ', len(_jpQes)+1)
    except:
      _jpQes = []
      print('jpQes LEN  :  ', len(_jpQes))
    for _i, jpQes in enumerate(_jpQes):
      Answer.append(['{}_{:02x}'.format(No, _i+1), jpQes, Ans])


  return Answer

def trans_googleCloud(qes, src='ja', to='en-US'):
  project_id = "mimblewimble-230908"
  location = 'global'

  parent = Trans.location_path(project_id, location)

  response = Trans.translate_text(
    parent=parent,
    contents=qes,
    mime_type='text/plain',
    source_language_code = src,
    target_language_code = to)
  for translation in response.translations:
    question = translation.translated_text
    break

  return question

def paraphrasing_word(text):
  payload = {
    "input_text" : text,
    "max_questions": 5
  }
  return qg.paraphrase(payload)

In [ ]:
Result = Paraphrasing(data)

In [ ]:
pd.DataFrame(Result, columns=data.columns.values).to_csv('ChatBot_OPEN_0325_paraphrasing.csv', sep=',', encoding='cp932')
! cp ChatBot_OPEN_0325_paraphrasing.csv /content/drive/"My Drive"/"Colab Notebooks"/data

In [ ]:
! ls

In [ ]:
!ls ChatBot_OPEN_0325_paraphrasing.csv